# Modélisation - Prix Électricité France (2020-2025)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import lightgbm as lgb
from statsmodels.tsa.statespace.sarimax import SARIMAX
import shap
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error, root_mean_squared_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

C:\Users\conta\AppData\Roaming\Python\Python314\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Préparation Modélisation

In [2]:
# Charger le dataset avec features
df_ml = pd.read_csv('../../data/processed/df_ml_france_2020_2025.csv', 
                    parse_dates=['utc_timestamp'], 
                    index_col='utc_timestamp')
print(f"Dataset chargé: {df_ml.shape}")
df_ml.head()

Dataset chargé: (51810, 71)


,biomass,gas,coal,oil,hydro_pumped,hydro_river,hydro_reservoir,nuclear,solar,waste,...,renewable_generation,total_generation,residual_load,price_delta,price_delta_pct,renewable_ratio,load_x_hour,temp_x_cloud,temp_x_load,wind_x_speed
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-08 00:00:00+00:00,333.0,4439.0,45.0,197.0,627.166667,4050.0,704.0,51810.0,0.0,224.0,...,5464.0,57274.0,2580.0,-1.62,-5.605536,0.091287,0.00,5.450512,432.487527,12456.368224
2020-01-08 01:00:00+00:00,333.0,4401.0,46.0,198.0,639.000000,3887.0,609.0,51120.0,0.0,222.0,...,5593.0,56713.0,1624.0,-1.12,-4.105572,0.095872,583.37,5.478322,418.524537,12369.026886
2020-01-08 02:00:00+00:00,334.0,4322.0,45.0,201.0,650.833333,3781.0,513.0,49726.0,0.0,218.0,...,5468.0,55194.0,576.0,-5.30,-20.259939,0.098044,1115.40,5.446155,396.326454,11521.244961
2020-01-08 03:00:00+00:00,333.0,4427.0,45.0,201.0,662.666667,3777.0,493.0,50202.0,0.0,217.0,...,5256.0,55458.0,-454.0,-0.51,-2.444871,0.095555,1650.12,5.517680,390.500199,10546.306963
2020-01-08 04:00:00+00:00,333.0,4879.0,45.0,200.0,674.500000,3944.0,709.0,51295.0,0.0,219.0,...,4981.0,56276.0,961.0,8.17,40.147420,0.087023,2289.48,5.452582,406.590911,9701.657077


In [3]:
# Split temporel (Test = 9 derniers mois 2025)
test_start_date = '2025-04-01'
train = df_ml[df_ml.index < test_start_date]
test = df_ml[df_ml.index >= test_start_date]

print(f"Train: {train.index.min()} → {train.index.max()} ({len(train):,} samples)")
print(f"Test: {test.index.min()} → {test.index.max()} ({len(test):,} samples)")


Train: 2020-01-08 00:00:00+00:00 → 2025-03-31 23:00:00+00:00 (45,745 samples)
Test: 2025-04-01 00:00:00+00:00 → 2025-12-12 18:00:00+00:00 (6,065 samples)


In [4]:
# Traitement outliers sur train uniquement
target = 'price_day_ahead'
Q1, Q3 = train[target].quantile([0.25, 0.75])
IQR = Q3 - Q1
train = train.copy()
train[target] = train[target].clip(Q1 - 1.5*IQR, Q3 + 1.5*IQR)

In [5]:
# Préparation X, y
target = 'price_day_ahead'

# Colonnes techniques à supprimer (y compris load_bin qui plante LightGBM)
drop_cols_technical = ['day_name', 'season_lbl', 'season', 'price_raw', 'load_bin', 'utc_timestamp']

# Colonnes cibles ou fuites potentielles (Leakage)
# On supprime TOUT ce qui contient 'price_day_ahead' SAUF si c'est un lag ou un rolling
drop_cols_leakage = [c for c in df_ml.columns if target in c and 'lag' not in c and 'rolling' not in c]

# Fusion des listes
drop_cols = list(set(drop_cols_technical + drop_cols_leakage))
# Filtrer pour ne garder que ce qui existe vraiment
drop_cols = [c for c in drop_cols if c in train.columns]

print(f"Colonnes supprimées : {len(drop_cols)}")
print(drop_cols)


Colonnes supprimées : 6
['price_day_ahead', 'season_lbl', 'load_bin', 'price_raw', 'season', 'day_name']


In [6]:

X_train, y_train = train.drop(columns=drop_cols), train[target]
X_test, y_test = test.drop(columns=drop_cols), test[target]

print(f"\nX_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")


X_train: (45745, 65)
X_test: (6065, 65)


### Modélisation LightGBM

In [7]:
def safe_mape(y_true, y_pred):
    mask = y_true > 1.0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else np.nan

# Baseline
print("\n LightGBM Baseline ")
model_base = lgb.LGBMRegressor(random_state=42, verbose=-1, n_jobs=-1)
model_base.fit(X_train, y_train)
preds_base = model_base.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, preds_base))
mae_base = mean_absolute_error(y_test, preds_base)
r2_base = r2_score(y_test, preds_base)
mape_base = safe_mape(y_test.values, preds_base)
print(f" RMSE: {rmse:.2f} | MAE: {mae_base:.2f} | R²: {r2_base:.3f} | MAPE: {mape_base:.2f}%")


 LightGBM Baseline 
 RMSE: 2.21 | MAE: 0.85 | R²: 0.997 | MAPE: 3.04%


In [8]:
# Optimisé
print("\n  LightGBM Optimisé ")
param_grid = {
                'learning_rate': [0.01, 0.05, 0.1],
                'num_leaves': [31, 50, 100],
                'max_depth': [-1, 5, 10],
                'n_estimators': [100, 200, 500]
            }
gbm = lgb.LGBMRegressor(random_state=42, 
                        verbose=-1, 
                        n_jobs=-1)
                        
tscv = TimeSeriesSplit(n_splits=3)
grid = GridSearchCV(gbm, param_grid, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=0)
grid.fit(X_train.iloc[-len(X_train)//2:], y_train.iloc[-len(y_train)//2:])

print(f"Hyperparamètres optimaux: {grid.best_params_}")



  LightGBM Optimisé 
Hyperparamètres optimaux: {'learning_rate': 0.05, 'max_depth': -1, 'n_estimators': 200, 'num_leaves': 100}


In [9]:

best_gbm = grid.best_estimator_
best_gbm.fit(X_train, y_train)
preds_optim = best_gbm.predict(X_test)

rmse_optim = root_mean_squared_error(y_test, preds_optim)
mae_optim = mean_absolute_error(y_test, preds_optim)
r2_optim = r2_score(y_test, preds_optim)
mape_optim = safe_mape(y_test.values, preds_optim)
print(f"RMSE: {rmse_optim:.2f} | MAE: {mae_optim:.2f} | R²: {r2_optim:.3f} | MAPE: {mape_optim:.2f}%")
print(f"Amélioration: {mae_base - mae_optim:.2f} €/MWh")

RMSE: 1.86 | MAE: 0.61 | R²: 0.998 | MAPE: 2.06%
Amélioration: 0.24 €/MWh


## Visualisations

In [10]:
layout_config = dict(
    font=dict(size=16, family='Arial'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgray', linecolor='black'),
    yaxis=dict(showgrid=True, gridcolor='lightgray', linecolor='black')
)

# Baseline
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, name='Réel', line=dict(color='#2E7D32', width=3)))
fig.add_trace(go.Scatter(x=y_test.index, y=preds_base, name='prédit', line=dict(color='red', width=2, dash='dot')))
fig.update_layout(
    title=dict(text=f'<b>LightGBM Baseline</b><br>MAE: {mae_base:.2f} €/MWh | R²: {r2_base:.3f}', font=dict(size=20)),
    xaxis_title='Date', yaxis_title='Prix (€/MWh)', height=600, width=1200, **layout_config
)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
# Optimisé
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, name='Réel', line=dict(color='#2E7D32', width=3)))
fig.add_trace(go.Scatter(x=y_test.index, y=preds_optim, name='prédit', line=dict(color='#D32F2F', width=2)))
fig.update_layout(
    title=dict(text=f'<b>LightGBM Optimisé</b><br>MAE: {mae_optim:.2f} €/MWh | R²: {r2_optim:.3f}', font=dict(size=20)),
    xaxis_title='Date', yaxis_title='Prix (€/MWh)', height=600, width=1200, **layout_config
)
fig.show()

In [ ]:
# Comparaison
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, name='Réel', line=dict(color='#2E7D32', width=3)))
fig.add_trace(go.Scatter(x=y_test.index, y=preds_base, name=f'Baseline (MAE={mae_base:.1f})', 
                         line=dict(color='#1976D2', width=1.5, dash='dot'), opacity=0.7))
fig.add_trace(go.Scatter(x=y_test.index, y=preds_optim, name=f'Optimisé (MAE={mae_optim:.1f})', 
                         line=dict(color='#D32F2F', width=2)))
fig.update_layout(
    title=dict(text='<b>Comparaison LightGBM</b>', font=dict(size=22)),
    xaxis_title='Date', yaxis_title='Prix (€/MWh)', height=700, width=1200,
    legend=dict(font=dict(size=14), x=0.02, y=0.98), **layout_config
)
fig.show()

In [ ]:
# Résidus
residuals_base = y_test - preds_base
residuals_optim = y_test - preds_optim

fig = make_subplots(rows=1, cols=2, subplot_titles=('Résidus Baseline', 'Résidus Optimisé'))
fig.add_trace(go.Scatter(x=y_test.index, y=residuals_base, mode='markers',
                         marker=dict(size=4, color='#1976D2', opacity=0.5)), row=1, col=1)
fig.add_trace(go.Scatter(x=y_test.index, y=residuals_optim, mode='markers',
                         marker=dict(size=4, color='#D32F2F', opacity=0.5)), row=1, col=2)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="black", row=1, col=2)
fig.update_layout(title="<b>Analyse des Résidus</b>", height=500, showlegend=False, font=dict(size=14))
fig.show()

## ANALYSE SHAP

In [ ]:
# coefficients SHAP
X_shap = X_test.iloc[:1000]
explainer = shap.TreeExplainer(best_gbm)
shap_values = explainer.shap_values(X_shap)


In [ ]:
# visualisation SHAP
shap_importance = np.abs(shap_values).mean(axis=0)

df_shap = pd.DataFrame({
    'Feature': X_shap.columns,
    'Importance': shap_importance
})

# Tri par ordre d'importance
df_shap = df_shap.sort_values(by='Importance', ascending=True)

# Visualisation 
fig = px.bar(df_shap, 
             x='Importance', 
             y='Feature', 
             orientation='h',
             title='<b>Importance des Features (SHAP Global)</b>',
             text_auto='.2f',  
             color='Importance',
             color_continuous_scale='Viridis')

fig.update_layout(
    height=800, 
    xaxis_title="Impact Moyen absolu sur le prix (€/MWh)",
    yaxis_title="",
    font=dict(size=12)
)

fig.show()

#### Modèle SARIMAX

#### Préparation des Données

In [ ]:
# Sélection des variables exogènes
exog_features = ['gas', 'coal', 'nuclear', 'solar', 'wind', 'biomass', 'waste', 'load', 'temperature', 'cloud_cover', 'wind_speed']
# Vérification qu'elles existent
available_exog = [c for c in exog_features if c in df_ml.columns]
print(f"Variables Exogènes retenues : {available_exog}")


Variables Exogènes retenues : ['gas', 'coal', 'nuclear', 'solar', 'wind', 'biomass', 'waste', 'load', 'temperature', 'cloud_cover', 'wind_speed']


#### Split en train/test

In [ ]:
# Split en train/test (Aligné sur X_train pour les dates)
train_exog = df_ml.loc[df_ml.index < test_start_date, available_exog].fillna(method='ffill').fillna(0)
test_exog = df_ml.loc[df_ml.index >= test_start_date, available_exog].fillna(method='ffill').fillna(0)

# Cible (avec Winsorization pour la stabilité du training)
train_target = df_ml.loc[df_ml.index < test_start_date, 'price_day_ahead'].copy()
test_target = df_ml.loc[df_ml.index >= test_start_date, 'price_day_ahead']

# Winsorization 
Q1 = train_target.quantile(0.25)
Q3 = train_target.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
train_target = train_target.clip(lower=lower_bound, upper=upper_bound)

print(f"Train Exog shape: {train_exog.shape}")
print(f"Test Exog shape: {test_exog.shape}")


Train Exog shape: (45745, 11)
Test Exog shape: (6065, 11)


#### Entraînement

In [ ]:
# Entraînement SARIMAX
model_sarimax = SARIMAX(train_target, 
                        exog=train_exog,
                        order=(1, 1, 1), 
                        seasonal_order=(1, 0, 1, 24),
                        enforce_stationarity=False, 
                        enforce_invertibility=False)

print("Entraînement en cours...")
fitted_sarimax = model_sarimax.fit(disp=False, maxiter=100)
print("Terminé")


Entraînement en cours...


In [ ]:
# Prédiction 
preds_sarimax = fitted_sarimax.forecast(steps=len(test_target), exog=test_exog)


In [ ]:
# Evaluation
mae_sarimax = mean_absolute_error(test_target, preds_sarimax)
rmse_sarimax = np.sqrt(mean_squared_error(test_target, preds_sarimax))
r2_sarimax = r2_score(test_target, preds_sarimax)
mape_sarimax = safe_mape(test_target.values, preds_sarimax.values)

print(f"MAE: {mae_sarimax:.2f} | RMSE: {rmse_sarimax:.2f} | R²: {r2_sarimax:.3f} | MAPE: {mape_sarimax:.2f}%")


In [ ]:
# Visualisation 
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_target.index, y=test_target, name='Réel', line=dict(color='#2E7D32', width=3)))
fig.add_trace(go.Scatter(x=test_target.index, y=preds_sarimax, name='ARIMAX (Multivarié)', line=dict(color='#FF6F00', width=2)))
fig.update_layout(title=f'ARIMAX (MAE: {mae_sarimax:.2f} €/MWh)', height=500)
fig.show()

#### Comparaison Finale

In [ ]:
print("\n" + "="*70)
print("RÉSUMÉ COMPARATIF")
print("="*70)
print(f"{'Modèle':<25} {'MAE':<15} {'R²':<10} {'MAPE (%)':<10}")
print("-"*70)
print(f"{'LightGBM Baseline':<25} {mae_base:<15.2f} {r2_base:<10.3f} {mape_base:<10.2f}")
print(f"{'LightGBM Optimisé':<25} {mae_optim:<15.2f} {r2_optim:<10.3f} {mape_optim:<10.2f}")
print(f"{'SARIMAX':<25} {mae_sarimax:<15.2f} {r2_sarimax:<10.3f} {mape_sarimax:<10.2f}")
print("="*70)

best = min([('LightGBM Baseline', mae_base), ('LightGBM Optimisé', mae_optim), ('SARIMAX', mae_sarimax)], key=lambda x: x[1])
print(f"\n Meilleur: {best[0]} (MAE = {best[1]:.2f} €/MWh)")


RÉSUMÉ COMPARATIF
Modèle                    MAE             R²         MAPE (%)  
----------------------------------------------------------------------
LightGBM Baseline         0.34            0.998      1.13      
LightGBM Optimisé         0.32            0.998      1.10      
SARIMAX                   102.44          -7.164     576.48    

 Meilleur: LightGBM Optimisé (MAE = 0.32 €/MWh)
